In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score, precision_score, accuracy_score,f1_score,confusion_matrix
from sklearn.model_selection import KFold
kf = KFold(n_splits= 5 )
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits= 5 , shuffle= True , random_state= 1 )
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
dec_tree = DecisionTreeClassifier()
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [25]:
df =pd.read_csv('/content/gdrive/My Drive/Hamoyec/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [6]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


### Preprocessing
Perform initial data preparation by converting the 'TotalCharges' column to numeric values and filling missing values with 0.

In [26]:
for i in range(0,len(df)):
  if df['TotalCharges'][i]==' ':
    df['TotalCharges'][i]=0.0
  else:
    df['TotalCharges'][i] = float(df['TotalCharges'][i])

<ipython-input-26-0245b3c6652d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TotalCharges'][i] = float(df['TotalCharges'][i])
<ipython-input-26-0245b3c6652d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TotalCharges'][i]=0.0


In [27]:
df['TotalCharges']=df['TotalCharges'].astype(float)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


Convert the 'Churn' column to binary values, where 'No' is mapped to 0 and 'Yes' is mapped to 1.

In [28]:
for i in range(0,len(df)):
  if df['Churn'][i]=='No':
    df['Churn'][i]=0
  else:
    df['Churn'][i] = 1

<ipython-input-28-07eafb63406c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Churn'][i]=0
<ipython-input-28-07eafb63406c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Churn'][i] = 1


In [29]:
df['Churn']=df['Churn'].astype(int)

Feature Engineering

In [30]:
categorical = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService','OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies','Contract', 'PaperlessBilling', 'PaymentMethod']
numerical = ['tenure', 'MonthlyCharges', 'TotalCharges']
# x_train, x_test, y_train, y_test = train_test_split(X, y, test_size= 0.3 , random_state= 0 )

In [33]:
df['gender'].unique()

array(['Female', 'Male'], dtype=object)

In [34]:
from sklearn.preprocessing import OneHotEncoder
encoder=OneHotEncoder(sparse_output=False)

In [38]:
f=encoder.fit_transform(df[categorical])
f.shape

(7043, 43)

In [39]:
for i in categorical:
  df[i]=encoder.fit_transform(df[i].values.reshape(-1,1))

In [56]:
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,0.0,1.0,1.0,0.0,-1.277445,1.0,1.0,0.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,-1.160323,-0.992611,0
1,5575-GNVDE,0.0,1.0,0.0,1.0,0.066327,1.0,0.0,1.0,1.0,...,1.0,0.0,1.0,1.0,1.0,0.0,1.0,-0.259629,-0.172165,0
2,3668-QPYBK,0.0,1.0,0.0,1.0,-1.236724,1.0,0.0,1.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,-0.362660,-0.958066,1
3,7795-CFOCW,0.0,1.0,0.0,1.0,0.514251,1.0,1.0,0.0,1.0,...,1.0,0.0,0.0,1.0,1.0,0.0,1.0,-0.746535,-0.193672,0
4,9237-HQITU,0.0,1.0,1.0,1.0,-1.236724,1.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.197365,-0.938874,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,0.0,1.0,0.0,0.0,-0.340876,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.665992,-0.127605,0
7039,2234-XADUH,0.0,1.0,1.0,0.0,1.613701,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.277533,2.242606,0
7040,4801-JZAZL,0.0,1.0,1.0,0.0,-0.870241,0.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,-1.168632,-0.852932,0
7041,8361-LTMKD,1.0,0.0,0.0,0.0,-1.155283,1.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.320338,-0.870513,1


In [40]:
cols=numerical+categorical

In [41]:
X=df[cols]
y=df['Churn'].astype(int)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size= 0.2 , random_state= 1)

Use scikit learn to train a random forest and extra trees classifier, and use xgboost and lightgbm to train an extreme boosting model and a light gradient boosting model. Use random_state = 1 for training all models and evaluate on the test set.

In [42]:
# importing the module
from sklearn.ensemble import ExtraTreesClassifier

# initializing the model
extra_classifier = ExtraTreesClassifier()
from sklearn.ensemble import RandomForestClassifier

# instantiate the classifier
random_classifier = RandomForestClassifier()


In [43]:
y_train.info()

<class 'pandas.core.series.Series'>
Int64Index: 5634 entries, 1814 to 5157
Series name: Churn
Non-Null Count  Dtype
--------------  -----
5634 non-null   int64
dtypes: int64(1)
memory usage: 88.0 KB


In [44]:
extra_classifier.fit(x_train,y_train)
preds=extra_classifier.predict(x_test)
accuracy_score(preds,y_test)

0.7771469127040455

In [45]:
random_classifier.fit(x_train,y_train)
preds=random_classifier.predict(x_test)
accuracy_score(preds,y_test)

0.7991483321504613

In [46]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
xgb =XGBClassifier()
lgb=LGBMClassifier()

In [47]:
xgb.fit(x_train,y_train)
preds=xgb.predict(x_test)
accuracy_score(preds,y_test)

0.794180269694819

In [48]:
lgb.fit(x_train,y_train)
preds=lgb.predict(x_test)
accuracy_score(preds,y_test)

[LightGBM] [Info] Number of positive: 1521, number of negative: 4113
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001678 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 615
[LightGBM] [Info] Number of data points in the train set: 5634, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.269968 -> initscore=-0.994785
[LightGBM] [Info] Start training from score -0.994785


0.8076650106458482